Use Environment of Fine tuning

In [ ]:
%pip install --quiet --upgrade langchain-text-splitters langchain langchain-community langgraph

In [ ]:
#%pip install U "langchain[google-genai]"

In [ ]:
#%pip install U "langchain[openai]"

If we are using for openAI - we need to change "langchain[google-genai]" to OpenAI's reference line

Refer link - https://python.langchain.com/docs/tutorials/rag/

### 1. Developing Agents from scratch without using any frame works

#### a. Creating and passing Langsmith API key

In [ ]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your Langsmith API key: ")

Alternate way to input API key is through environment (.env) package

In [ ]:
!pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")

#### b. Using Opening AI LLM model > Code is taken  > from Components for RAG tutorial
link - https://python.langchain.com/docs/tutorials/rag/

In [ ]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [ ]:
# TO check if the LLM is working, we can invoke it with a simple question.
llm.invoke("Who is the PM of India")

#### c. Selecting the embedding model related to the LLM initiated > To convert the input data to correct embedding format

In [ ]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### d. Now to select the memory to save the embeddings > TO save it in RAM or HardDisk

In [ ]:
#Saving embeddings to a hard disk file
#%pip install -qU langchain-chroma

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="RAG_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)



Since the above is saved in hard disk, we can load it back later. To view the information we use DBeaver

#### e. Use Cases

##### i.  --> We are referring to the information in the link, apply webscraping and try to apply RAG on its content

link : https://lilianweng.github.io/posts/2023-06-23-agent/

If we want to load a different file - refer to the below documentation from langchain to copy the syntax

link : https://python.langchain.com/docs/integrations/document_loaders/

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict


If we use different file - refer to the below documentation from langchain to copy the syntax

link : https://python.langchain.com/docs/integrations/document_loaders/

In [ ]:

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header") 
            # These information/tagging are used to filter the HTML content from the website | This would be dynamic for each website, we need to identify the same from inspecting content and website to extract
            # You can also use id_ or other attributes to filter the content
        )
    ),
)
docs = loader.load()

#Chunking the entire content in docs to smaller chunks to ensure the window is less than 128k tokens
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200) 
all_splits = text_splitter.split_documents(docs)

# Index chunks or Embedding them into the vector store
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
# N.B. for non-US LangSmith endpoints, you may need to specify
# api_url="https://api.smith.langchain.com" in hub.pull.
prompt = hub.pull("rlm/rag-prompt")


# Define state for application > 
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"]) # Matching or identifying the distance wrt question | We can also use FIAS for this
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"]) # Joining the content of all the documents retrieved to form a single context with next line as separator
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
#Entire Page content based on HTML tagging mentioned
docs

##### ii. PDF file

If we want to load a different file - refer to the below documentation from langchain to copy the syntax

link : https://python.langchain.com/docs/integrations/document_loaders/

In [ ]:
%pip install -qU pypdf

Selecting the PDF file to apply RAG

In [ ]:
file_path = (
    "../../docs/integrations/document_loaders/example_data/layout-parser-paper.pdf"
)

Loading the data

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path)
docs = []
async for page in loader.alazy_load():
    docs.append(page)


print(f"{docs[0].metadata}\n")
print(docs[0].page_content)

Everything else from docs are the same as used previously

In [ ]:
#Chunking the entire content in docs to smaller chunks to ensure the window is less than 128k tokens
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200) 
all_splits = text_splitter.split_documents(docs)

# Index chunks or Embedding them into the vector store
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
# N.B. for non-US LangSmith endpoints, you may need to specify
# api_url="https://api.smith.langchain.com" in hub.pull.
prompt = hub.pull("rlm/rag-prompt")


# Define state for application > 
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"]) # Matching or identifying the distance wrt question | We can also use FIAS for this
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"]) # Joining the content of all the documents retrieved to form a single context with next line as separator
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()